In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split



# Basisstijl voor plots

plt.rcParams["figure.figsize"] = (10, 6)

plt.rcParams["axes.grid"] = True



# ============================================================================

# STAP 1: DATA INLADEN EN EERSTE VERKENNING

# ============================================================================

print("="*80)

print("STAP 1: DATA INLADEN EN EERSTE VERKENNING")

print("="*80)



# Laad de dataset

df = pd.read_csv("ML-sleep_health_lifestyle_dataset_5000_target82.csv")



print(f"\nDataset geladen met {df.shape[0]} rijen en {df.shape[1]} kolommen")

print("\nEerste 5 rijen:")

display(df.head())

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# ============================================================================
# STAP 2: DATA INSPECTIE EN KWALITEITSCONTROLE
# ============================================================================
print("\n" + "="*80)
print("STAP 2: DATA INSPECTIE EN KWALITEITSCONTROLE")
print("="*80)

print("\nInformatie over datatypes en geheugengebruik:")
df.info()

print("\n" + "-"*80)
print("CONTROLE OP ONTBREKENDE WAARDEN")
print("-"*80)
missing_values = df.isnull().sum()
missing_pct = 100 * df.isnull().sum() / len(df)
missing_table = pd.DataFrame({
    'Aantal Missing': missing_values,
    'Percentage': missing_pct
})
print(missing_table[missing_table['Aantal Missing'] > 0])

if missing_table['Aantal Missing'].sum() == 0:
    print("\n✓ Geen ontbrekende waarden gevonden in de dataset")
    print("Dit is gunstig voor modelontwikkeling omdat we geen imputatiestrategieën nodig hebben.")
else:
    print("\n⚠ Er zijn ontbrekende waarden die behandeld moeten worden")

print("\n" + "-"*80)
print("BESCHRIJVENDE STATISTIEKEN - NUMERIEKE VARIABELEN")
print("-"*80)
display(df.describe())





In [ ]:
# ============================================================================
# STAP 3: TARGET VARIABELE ANALYSE
# ============================================================================
print("\n" + "="*80)
print("STAP 3: TARGET VARIABELE ANALYSE")
print("="*80)

print("\nDe target variabele 'Sleep Disorder' vormt de basis voor ons classificatieprobleem.")
print("We onderzoeken de verdeling van klassen om te bepalen of we te maken hebben met")
print("class imbalance en of dit speciale aandacht vereist tijdens modeltraining.\n")

# Vervang NaN in Sleep Disorder met 'None'
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

print("Verdeling van Sleep Disorder:")
class_distribution = df['Sleep Disorder'].value_counts()
print(class_distribution)
print(f"\nPercentages:")
print(df['Sleep Disorder'].value_counts(normalize=True) * 100)

plt.figure(figsize=(8, 5))
df['Sleep Disorder'].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c', '#3498db'])
plt.title("Verdeling van Sleep Disorder Klassen", fontsize=14, fontweight='bold')
plt.xlabel("Klasse")
plt.ylabel("Aantal observaties")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n" + "-"*80)
print("INTERPRETATIE VAN KLASSENBALANS")
print("-"*80)

# Bereken imbalance ratio
min_class = class_distribution.min()
max_class = class_distribution.max()
imbalance_ratio = max_class / min_class

print(f"\nImbalance ratio (grootste/kleinste klasse): {imbalance_ratio:.2f}")

if imbalance_ratio < 1.5:
    print(" De klassen zijn redelijk gebalanceerd. Standaard modeltraining is geschikt.")
elif imbalance_ratio < 3:
    print("Er is enige klassenonevenwichtigheid. Overweeg stratified sampling en")
    print("  class_weight='balanced' parameter bij sommige modellen.")
else:
    print("Significante klassenonevenwichtigheid gedetecteerd!")
    print("  Overweeg: SMOTE, class weighting, of stratified k-fold cross-validation.")


In [ ]:

# ============================================================================
# STAP 4: MULTICLASS VS BINARY CLASSIFICATIE - ONDERBOUWING
# ============================================================================
print("\n" + "="*80)
print("STAP 4: MODELTYPE KEUZE - MULTICLASS VS BINARY CLASSIFICATIE")
print("="*80)

print("""
RATIONALE VOOR MULTICLASS CLASSIFICATIE:
-----------------------------------------

Onze target variabele heeft drie categorieën:
1. None (geen slaapstoornis)
2. Insomnia (slapeloosheid)
3. Sleep Apnea (slaapapneu)

WAAROM MULTICLASS IN PLAATS VAN BINARY?
========================================

1. KLINISCHE RELEVANTIE:
   • Insomnia en Sleep Apnea hebben verschillende oorzaken, symptomen en behandelingen
   • Een binair model (wel/geen stoornis) zou deze cruciale distinctie verliezen
   • Voor medisch personeel is het essentieel om het TYPE stoornis te identificeren

2. BEHANDELINGSIMPLICATIES:
   • Insomnia → vaak cognitieve gedragstherapie, slaaphygiëne, medicatie
   • Sleep Apnea → CPAP-apparaat, gewichtsreductie, operatieve ingrepen
   • De aanpak verschilt fundamenteel

3. DIAGNOSTISCHE WAARDE:
   • Verschillende risicoprofielen: Sleep Apnea correleert met BMI en hartslag,
     Insomnia vaak met stress en levensstijlfactoren
   • Een multiclass model kan deze subtiele patronen onderscheiden

4. MODELCOMPLEXITEIT VS INFORMATIEBEHOUD:
   • Trade-off: multiclass is complexer, maar behoudt essentiële informatie
   • In medische context weegt informatieverlies zwaarder dan modelcomplexiteit


""")


In [ ]:

# ============================================================================
# STAP 5: FEATURE TYPE IDENTIFICATIE EN CATEGORISATIE
# ============================================================================
print("\n" + "="*80)
print("STAP 5: FEATURE TYPE IDENTIFICATIE")
print("="*80)

# Numerieke kolommen
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Verwijder Person ID uit numerieke features (is geen predictive feature)
num_cols = [col for col in num_cols if col != 'Person ID']

# Categorische kolommen
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
# Verwijder target uit categorische features
cat_feature_cols = [col for col in cat_cols if col != 'Sleep Disorder']

print(f"\nNumerieke features ({len(num_cols)}):")
for col in num_cols:
    print(f"  • {col}")

print(f"\nCategorische features ({len(cat_feature_cols)}):")
for col in cat_feature_cols:
    unique_values = df[col].nunique()
    print(f"  • {col} ({unique_values} unieke waarden)")
    print(f"    Waarden: {df[col].unique()[:5].tolist()}")


In [ ]:
# ============================================================================
# STAP 6: TRAIN-TEST SPLIT - DATA PARTITIONERING
# ============================================================================
print("\n" + "="*80)
print("STAP 6: TRAIN-TEST SPLIT")
print("="*80)

print("""
DOEL VAN TRAIN-TEST SPLIT:
===========================

We splitsen de data in twee sets:
1. TRAINING SET: Model leert patronen uit deze data
2. TEST SET: Model wordt geëvalueerd op ongeziene data

Dit voorkomt OVERFITTING en geeft realistische performantie schatting.

KRITIEK: SPLIT VOOR ALLE TRANSFORMATIES!
=========================================

We splitsen op de ONBEWERKTE data (na feature identificatie).
Alle bewerkingen (outlier behandeling, encoding, scaling) gebeuren DAARNA,
APART op train en test set.

WAAROM?
- Voorkomt data leakage (test info lekt niet naar train)
- Scaler fit op train, transform op test
- Encoder fit op train, transform op test
- Outlier grenzen bepaald op train, toegepast op test

SPLIT RATIO OVERWEGINGEN:
==========================

OPTIES:
- 90/10: Maximaal trainingsdata, maar kleine test set (minder betrouwbare metrics)
- 80/20: Goede balans (ONZE KEUZE)
- 70/30: Meer test data, maar minder trainingsdata
- 60/40: Bij zeer kleine datasets

ONZE KEUZE: 80/20 SPLIT
========================

Rationale:
- Dataset heeft ~374 samples → 80% = ~299 train, 20% = ~75 test
- 80% training is genoeg voor model learning
- 20% test is acceptabel voor betrouwbare evaluatie (hoewel aan de kleine kant)
- Breed geaccepteerd in ML literatuur (Hastie et al., 2009)

STRATIFIED SAMPLING:
====================

stratify=y zorgt dat de klassenverhouding gelijk blijft in train én test.

Voorbeeld: Als originele data 50% None, 30% Insomnia, 20% Sleep Apnea heeft,
dan heeft ZOWEL train ALS test deze verhoudingen.

WAAROM STRATIFICATION CRUCIAAL IS:
- Voorkomt dat één klasse oververtegenwoordigd is in test set
- Zorgt voor representatieve evaluatie
- Essentieel bij (lichte) class imbalance

RANDOM STATE:
=============
random_state=42 zorgt voor reproduceerbaarheid.
Elke run geeft identieke split → belangrijk voor:
- Vergelijking tussen modellen
- Rapportage van resultaten
- Samenwerking in team
""")

print("\n" + "-"*80)
print("UITVOEREN VAN SPLIT OP ONBEWERKTE DATA")
print("-"*80)

# Scheiding van features en target VOOR encoding/scaling
X = df.drop(['Sleep Disorder', 'Person ID'], axis=1)  # ✓ originele data
y = df['Sleep Disorder']  # nog als string

print(f"\nOriginele dataset:")
print(f"  • X shape: {X.shape}")
print(f"  • y shape: {y.shape}")

# Split op onbewerkte data
X_train, X_test, y_train, y_test = train_test_split(
    X,  # ✓ onbewerkte features
    y,  # ✓ onbewerkte target (nog strings)
    test_size=0.20,
    random_state=42,
    stratify=y
)

print(f"\nTRAINING SET:")
print(f"  • X_train shape: {X_train.shape}")
print(f"  • y_train shape: {y_train.shape}")
print(f"  • Percentage: {100 * len(X_train) / len(X):.1f}%")

print(f"\nTEST SET:")
print(f"  • X_test shape: {X_test.shape}")
print(f"  • y_test shape: {y_test.shape}")
print(f"  • Percentage: {100 * len(X_test) / len(X):.1f}%")

print("\n" + "-"*80)
print("VERIFICATIE VAN STRATIFICATIE")
print("-"*80)

print("\nKlassenverdeling in ORIGINELE data:")
print(y.value_counts(normalize=True).sort_index())

print("\nKlassenverdeling in TRAINING set:")
print(y_train.value_counts(normalize=True).sort_index())

print("\nKlassenverdeling in TEST set:")
print(y_test.value_counts(normalize=True).sort_index())

# Visualisatie van stratificatie
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

y.value_counts(normalize=True).sort_index().plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Original Data')
axes[0].set_ylabel('Proportion')
axes[0].set_xlabel('Class')
axes[0].set_ylim(0, 0.6)

y_train.value_counts(normalize=True).sort_index().plot(kind='bar', ax=axes[1], color='lightgreen')
axes[1].set_title('Training Set (80%)')
axes[1].set_ylabel('Proportion')
axes[1].set_xlabel('Class')
axes[1].set_ylim(0, 0.6)

y_test.value_counts(normalize=True).sort_index().plot(kind='bar', ax=axes[2], color='lightcoral')
axes[2].set_title('Test Set (20%)')
axes[2].set_ylabel('Proportion')
axes[2].set_xlabel('Class')
axes[2].set_ylim(0, 0.6)

plt.suptitle('Verificatie Stratified Split - Klassenverhouding blijft gelijk', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✓ Stratificatie succesvol: klassenverhouding identiek in train en test")

In [ ]:
# ============================================================================
# STAP 7: EXPLORATIEVE DATA ANALYSE (EDA)
# ============================================================================
print("\n" + "="*80)
print("STAP 7: EXPLORATIEVE DATA ANALYSE (EDA)")
print("="*80)

print("""
BELANGRIJK: EDA wordt uitgevoerd op de TRAINING SET
====================================================

Na de train-test split in stap 6, voeren we alle analyses uit op de training data.
De test set blijft 'ongezien' om data leakage te voorkomen.

Dit betekent:
- Distributies → berekend op train
- Correlaties → berekend op train  
- Outlier grenzen → bepaald op train (en toegepast op test in stap 8)
- Feature statistics → berekend op train
""")

# Combineer X_train en y_train voor analyse
df_train = X_train.copy()
df_train['Sleep Disorder'] = y_train.values

print(f"\nTraining set voor EDA: {df_train.shape[0]} samples × {df_train.shape[1]} kolommen")

# ============================================================================
# FEATURE IMPORTANCE - WAAROM ZIJN DEZE FEATURES BELANGRIJK?
# ============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE - KLINISCHE RATIONALE")
print("="*80)

print("""
WAAROM ZIJN DEZE FEATURES BELANGRIJK VOOR SLAAPSTOORNIS CLASSIFICATIE?
========================================================================

1. SLEEP DURATION (Slaapduur) 
   • Primaire indicator van slaapstoornissen
   • Insomnia: <6 uur per nacht (chronisch slaaptekort)
   • Sleep Apnea: vaak normale duur, maar gefragmenteerde slaap
   • Verwachting: Duidelijk verschil tussen klassen

2. QUALITY OF SLEEP (Slaapkwaliteit) 
   • Subjectieve maat (self-reported, schaal 1-10)
   • Capteert ervaren slaapkwaliteit die objectieve metingen kunnen missen
   • Lage score bij BEIDE insomnia en sleep apnea
   • Verwachting: Sterkste predictor voor aanwezigheid slaapstoornis

3. STRESS LEVEL (Stressniveau) - 
   • Chronische stress → verhoogd cortisol → verstoord circadiaans ritme
   • Bidirectionele relatie: stress → slechte slaap → meer stress
   • Vooral relevant voor insomnia (hyperarousal)
   • Verwachting: Hoog bij insomnia, gemiddeld bij sleep apnea

4. PHYSICAL ACTIVITY LEVEL (Fysieke Activiteit) - 
   • Regelmatige beweging verbetert slaapkwaliteit (adenosine opbouw)
   • Te weinig activiteit → verstoorde slaap-waak cyclus
   • Te veel (overtraining) → verhoogde cortisol
   • Verwachting: Lagere waarden geassocieerd met slaapstoornissen

5. HEART RATE (Hartslag) -
   • Verhoogd bij sleep apnea: repetitieve zuurstoftekort episodes → sympathische activatie
   • Normaal tot laag bij insomnia (compensatiemechanisme)
   • Discriminerende feature: kan sleep apnea onderscheiden van insomnia
   • Verwachting: Significant hogere HR bij sleep apnea patiënten

6. DAILY STEPS (Dagelijkse Stappen) -
   • Proxy voor algemene fysieke activiteit en lifestyle
   • Correleert met Physical Activity Level (mogelijke multicollineariteit)
   • Sedentair gedrag geassocieerd met slaapproblemen
   • Verwachting: Lagere stappen bij beide slaapstoornissen

7. BMI (Body Mass Index)
   • Obesitas (BMI >30) = grootste risicofactor voor obstructieve sleep apnea
   • Mechanisme: verhoogd weefsel in luchtwegen → obstructie
   • Minder relevant voor insomnia (meer psychologische factoren)
   • Verwachting: Sterk verhoogd specifiek in sleep apnea groep

8. BLOOD PRESSURE (Bloeddruk) - 
   • Hypertensie sterk geassocieerd met sleep apnea
   • Chronische zuurstoftekort → chronische sympathische activatie
   • Kan ook verhoogd zijn bij chronische stress (insomnia)
   • Verwachting: Hogere waarden bij sleep apnea, ook mogelijk bij insomnia

9. AGE (Leeftijd) -
   • Prevalentie sleep apnea neemt toe met leeftijd (weefselverlies, verminderde spierspanning)
   • Insomnia prevalentie meer variabel over levensfasen
   • Hormonale veranderingen (menopauze) beïnvloeden slaap
   • Verwachting: Sleep apnea groep gemiddeld ouder

10. GENDER (Geslacht) -
    • Mannen: 2-3x hogere kans op sleep apnea (anatomische verschillen)
    • Vrouwen: hogere prevalentie insomnia (hormonale factoren, stress)
    • Belangrijke stratificatie variabele
    • Verwachting: Gender als moderator voor type slaapstoornis

11. OCCUPATION (Beroep) - 
    • Shift werk → verstoord circadiaans ritme → insomnia
    • Stress-intensive beroepen → hogere insomnia prevalentie
    • Sedentair werk → risico op obesitas → sleep apnea
    • Verwachting: Bepaalde beroepen correleren met specifieke stoornissen

12. BMI CATEGORY (BMI Categorie) - 
    • Ordinale versie van BMI (Normal/Overweight/Obese)
    • Mogelijk redundant → overweeg feature selection
    • Overweight/Obese: directe link met sleep apnea
    • Verwachting: Mogelijk te verwijderen vanwege multicollineariteit met BMI

SAMENVATTING TOP PREDICTORS:
- Quality of Sleep: algemene indicator voor ALLE slaapstoornissen
- BMI: specifiek voor sleep apnea
- Heart Rate: discrimineert tussen sleep apnea vs insomnia
- Sleep Duration: algemene indicator voor ernst van problematiek
- Stress Level: specifiek voor insomnia
""")

# ============================================================================
# GEMIDDELDE FEATURE WAARDEN PER SLEEP DISORDER KLASSE
# ============================================================================
print("\n" + "="*80)
print("PER-KLASSE ANALYSE - VERSCHILLEN TUSSEN GROEPEN")
print("="*80)

print("\nDeze analyse toont gemiddelde feature waarden per Sleep Disorder klasse.")
print("Grote verschillen tussen klassen duiden op discriminerende features.\n")

for col in num_cols:
    print("\n" + "-"*80)
    print(f"FEATURE: {col}")
    print("-"*80)
    
    # Bereken statistics per klasse
    class_stats = df_train.groupby('Sleep Disorder')[col].agg([
        ('Mean', 'mean'),
        ('Std', 'std'),
        ('Min', 'min'),
        ('Max', 'max'),
        ('Count', 'count')
    ]).round(2)
    
    display(class_stats)
    
    # Interpretatie
    means = df_train.groupby('Sleep Disorder')[col].mean()
    max_class = means.idxmax()
    min_class = means.idxmin()
    difference = means.max() - means.min()
    
    print(f"\n INTERPRETATIE:")
    print(f"  • Hoogste gemiddelde: '{max_class}' = {means[max_class]:.2f}")
    print(f"  • Laagste gemiddelde: '{min_class}' = {means[min_class]:.2f}")
    print(f"  • Verschil: {difference:.2f} ({(difference/means.mean())*100:.1f}% van gemiddelde)")
    
    # Conclusie
    if difference / means.mean() > 0.2:  # >20% verschil
        print(f"  ✓ STERKE DISCRIMINERENDE FEATURE (>20% verschil)")
    elif difference / means.mean() > 0.1:  # 10-20% verschil
        print(f"  → Matige discriminerende feature (10-20% verschil)")
    else:
        print(f"  ⚠ Zwakke discriminerende feature (<10% verschil)")

# ============================================================================
# DISTRIBUTIE VAN NUMERIEKE VARIABELEN
# ============================================================================
print("\n" + "="*80)
print("DISTRIBUTIE VAN NUMERIEKE VARIABELEN (TRAINING SET)")
print("="*80)

# Maak histogrammen van TRAIN data
axes = df_train[num_cols].hist(figsize=(15, 12), bins=20, edgecolor='black')
plt.suptitle("Histogrammen van Numerieke Variabelen (Training Set)", 
             fontsize=16, fontweight='bold')

try:
    ax_list = axes.flatten()
except Exception:
    ax_list = [axes] if hasattr(axes, 'get_axes') else list(axes)
    
for ax in ax_list:
    ax.set_xlabel('Waarde')
    ax.set_ylabel('Frequentie')
    
plt.tight_layout()
plt.show()

print("""
INTERPRETATIE VAN DISTRIBUTIES:
================================

- Sleep Duration: 
  - Normalish verdeling rond 7-8 uur (verwacht patroon)
  - Mogelijk bimodaal (twee pieken: normale slapers vs insomnia)
  
- Quality of Sleep: 
  - Concentratie rond hogere waarden (7-9)
  - Linker staart: slaapstoornis patiënten
  
- Stress Level: 
  - Spreiding over gehele schaal (1-10)
  - Mogelijk uniform of licht rechts-scheef
  
- Physical Activity Level: 
  - Variatie in activiteitsniveaus
  - Check of correlatie met Daily Steps (multicollineariteit)
  
- Heart Rate: 
  - Concentratie rond 70-80 bpm (normaal rustritme)
  - Rechter staart: mogelijk sleep apnea patiënten
  
- Daily Steps: 
  - Rechtse scheefheid (veel lage waarden, enkele zeer hoge)
  - Extreme waarden (>15000) mogelijk tracking fouten of atleten
  
- BMI:
  - Rechts-scheve verdeling (normale populatie + obese groep)
  - Verwacht patroon voor algemene populatie
  
- Age:
  - Spreiding over volwassen leeftijden
  - Check of ouderen oververtegenwoordigd in sleep apnea groep
""")

# ============================================================================
# CORRELATIE ANALYSE
# ============================================================================
print("\n" + "="*80)
print("CORRELATIE ANALYSE (TRAINING SET)")
print("="*80)

print("\nCorrelatiematrix geeft inzicht in lineaire relaties tussen variabelen.")
print("Hoge correlaties tussen features kunnen wijzen op multicollineariteit.\n")

plt.figure(figsize=(12, 10))
correlation_matrix = df_train[num_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt='.2f', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title("Correlatie Heatmap - Numerieke Variabelen (Training Set)", 
          fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

# Identificeer sterke correlaties (|r| > 0.7, exclusief diagonaal)
print("\nSterk gecorreleerde variabelen (|r| > 0.7):")
strong_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            strong_correlations.append({
                'Var1': correlation_matrix.columns[i],
                'Var2': correlation_matrix.columns[j],
                'Correlation': correlation_matrix.iloc[i, j]
            })

if strong_correlations:
    print("\n⚠️ MULTICOLLINEARITEIT GEDETECTEERD:")
    for corr in strong_correlations:
        print(f"  • {corr['Var1']} ↔ {corr['Var2']}: r = {corr['Correlation']:.3f}")
    print("\n💡 AANBEVELING: Overweeg één van deze features te verwijderen in feature selection fase.")
else:
    print("  ✓ Geen sterke correlaties gevonden (geen multicollineariteit issues)")

# ============================================================================
# PAIRPLOT - RELATIES TUSSEN BELANGRIJKE VARIABELEN
# ============================================================================
print("\n" + "="*80)
print("PAIRPLOT - RELATIES TUSSEN BELANGRIJKE VARIABELEN (TRAINING SET)")
print("="*80)

# Selecteer subset voor pairplot (anders te groot)
cols_for_pairplot = [
    'Sleep Duration',
    'Quality of Sleep',
    'Stress Level',
    'Physical Activity Level',
    'Heart Rate',
    'Daily Steps'
]
cols_for_pairplot = [c for c in cols_for_pairplot if c in df_train.columns]

print(f"\nPairplot van {len(cols_for_pairplot)} belangrijkste variabelen")
print("Dit helpt om non-lineaire relaties en clusters te identificeren.")
print("Kleuren tonen verschillende Sleep Disorder klassen.\n")

sns.pairplot(df_train[cols_for_pairplot + ['Sleep Disorder']], 
             hue='Sleep Disorder', diag_kind='kde', palette='Set2')
plt.suptitle("Pairplot - Relaties tussen Features (Training Set)", 
             y=1.01, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("""
INTERPRETATIE PAIRPLOT:
======================

LET OP:
- Clusters per kleur: duidelijke scheiding tussen klassen = goede feature
- Overlap tussen kleuren: moeilijk te onderscheiden klassen
- Diagonaal (KDE plots): distributie per klasse
- Off-diagonal scatter plots: bivariate relaties

VERWACHT:
- BMI vs Heart Rate: mogelijk cluster voor sleep apnea (hoog BMI + hoge HR)
- Quality of Sleep vs Sleep Duration: positieve correlatie
- Stress Level vs Quality of Sleep: negatieve correlatie
""")

print("\n" + "="*80)
print("✓ EDA VOLTOOID OP TRAINING SET")
print("="*80)
print("\nBelangrijkste bevindingen worden gebruikt voor:")
print("  • Outlier behandeling (stap 8)")
print("  • Feature selection (stap 11)")
print("  • Model keuze en interpretatie")

In [ ]:
# ============================================================================
# STAP 8: OUTLIER DETECTIE EN BEHANDELING - GEFUNDEERDE AANPAK 
# ============================================================================
print("\n" + "="*80)
print("STAP 8: OUTLIER DETECTIE EN BEHANDELING")
print("="*80)

print("""
METHODOLOGIE: IQR (INTERQUARTILE RANGE) METHODE
================================================

De IQR-methode definieert outliers als waarden die buiten de volgende grenzen vallen:
- Lower bound = Q1 - 1.5 × IQR
- Upper bound = Q3 + 1.5 × IQR

KRITISCHE VRAAG: MOETEN WE OUTLIERS VERWIJDEREN?
==================================================

In dit project verwijderen we outliers niet automatisch. In plaats daarvan passen we een conservatieve
capping/winsorization toe: zeer hoge of lage waarden worden begrensd tot een minimum of maximum.
Op deze manier behouden we alle observaties (belangrijk in medische datasets), maar verminderen
we de invloed van onrepresentatieve uitschieters op het model.

BELANGRIJK: DATA LEAKAGE PREVENTIE
===================================

We berekenen outlier grenzen (Q1, Q3, IQR) ALLEEN op TRAINING data.
Deze grenzen worden vervolgens toegepast op ZOWEL train ALS test data.

Werkwijze:
1. Bereken Q1, Q3, IQR op X_train
2. Bepaal lower/upper bounds op basis van train statistics
3. Pas capping toe op X_train met deze grenzen
4. Pas DEZELFDE grenzen toe op X_test (geen herberekening!)

Dit voorkomt data leakage: test set informatie beïnvloedt niet de train set.
""")

# ============================================================================
# STAP 8A: OUTLIER DETECTIE OP TRAINING SET
# ============================================================================
print("\n" + "-"*80)
print("OUTLIER IDENTIFICATIE PER VARIABELE (TRAINING SET)")
print("-"*80)

# Maak kopieën om originele data te behouden
X_train_clean = X_train.copy()
X_test_clean = X_test.copy()

# Identificeer numerieke kolommen in X_train
num_cols_train = X_train_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nNumerieke features in training set: {len(num_cols_train)}")
print(f"Features: {num_cols_train}\n")

# Bereken outlier statistics op TRAIN data
outlier_info = {}
outlier_bounds = {}  # Opslaan voor toepassen op test set

for col in num_cols_train:
    # Bereken op TRAIN
    q1 = X_train_clean[col].quantile(0.25)
    q3 = X_train_clean[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    # Identificeer outliers in train
    mask = (X_train_clean[col] < lower) | (X_train_clean[col] > upper)
    outlier_count = int(mask.sum())
    outlier_pct = 100 * outlier_count / len(X_train_clean)
    
    # Opslaan voor rapportage
    outlier_info[col] = {
        'count': outlier_count,
        'pct': float(outlier_pct),
        'lower': float(lower),
        'upper': float(upper),
        'Q1': float(q1),
        'Q3': float(q3),
        'IQR': float(iqr)
    }
    
    # Opslaan bounds voor test set
    outlier_bounds[col] = {'lower': lower, 'upper': upper}

outlier_summary = pd.DataFrame.from_dict(outlier_info, orient='index')
outlier_summary = outlier_summary.sort_values('pct', ascending=False)

print("\nOutlier Samenvatting Training Set (gesorteerd op percentage):")
display(outlier_summary)

print("\n" + "-"*80)
print("VISUALISATIE: BOXPLOTS VOOR OUTLIER DETECTIE (TRAINING SET)")
print("-"*80)

n_cols_plot = len(num_cols_train)
n_rows = (n_cols_plot + 2) // 3  # Bereken aantal rijen nodig
n_cols = min(3, n_cols_plot)

plt.figure(figsize=(16, 4 * n_rows))
for i, col in enumerate(num_cols_train, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.boxplot(x=X_train_clean[col], color='skyblue')
    title = f"{col}\n({outlier_summary.loc[col, 'count']:.0f} outliers, {outlier_summary.loc[col, 'pct']:.1f}%)"
    plt.title(title, fontsize=10)
    plt.xlabel('')
plt.suptitle("Boxplots - Outlier Detectie (Training Set)", fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# ============================================================================
# STAP 8B: OUTLIER BEHANDELING - CAPPING/WINSORIZATION
# ============================================================================
print("\n" + "="*80)
print("OUTLIER BEHANDELING STRATEGIE - GEFUNDEERDE BESLISSINGEN (CAPPING)")
print("="*80)

print("""
PRINCIPE: CONSERVATIEVE AANPAK
=============================

- We verwijderen geen rijen puur op basis van IQR-outliers.
- We gebruiken winsorization/clipping (capping) voor sterk scheve/dispersed variabelen
  en voor duidelijk onrealistische waarden.
- Dit behoudt klinisch relevante extreme cases, maar vermindert de invloed van meetfouten
  en ongebruikelijke uitschieters op modeltraining.

IMPLEMENTATIE:
==============
1. Bepaal capping strategie per feature op basis van domeinkennis
2. Bereken grenzen op TRAIN set
3. Pas toe op TRAIN set
4. Pas DEZELFDE grenzen toe op TEST set
""")

print("\n" + "-"*80)
print("IMPLEMENTATIE VAN BEHANDELING (CAPPING / WINSORIZATION)")
print("-"*80)

# Strategie 1: Winsorization voor Daily Steps (1% - 99%)
if 'Daily Steps' in X_train_clean.columns:
    print("\n1. DAILY STEPS - Winsorization / Capping (1% - 99%)")
    print("-" * 40)
    
    # TRAIN: bereken percentiles
    low_p_train = X_train_clean['Daily Steps'].quantile(0.01)
    high_p_train = X_train_clean['Daily Steps'].quantile(0.99)
    
    # Toon originele ranges
    original_min_train = X_train_clean['Daily Steps'].min()
    original_max_train = X_train_clean['Daily Steps'].max()
    original_min_test = X_test_clean['Daily Steps'].min()
    original_max_test = X_test_clean['Daily Steps'].max()
    
    print(f"   TRAIN originele range: {original_min_train:.0f} - {original_max_train:.0f} stappen")
    print(f"   TEST originele range: {original_min_test:.0f} - {original_max_test:.0f} stappen")
    print(f"   Capping grenzen (van train): {low_p_train:.0f} - {high_p_train:.0f} stappen")
    
    # Pas capping toe op TRAIN
    X_train_clean['Daily Steps'] = np.clip(X_train_clean['Daily Steps'], low_p_train, high_p_train)
    
    # Pas DEZELFDE grenzen toe op TEST
    X_test_clean['Daily Steps'] = np.clip(X_test_clean['Daily Steps'], low_p_train, high_p_train)
    
    print(f"   ✓ Capping toegepast op train ({len(X_train_clean)} samples)")
    print(f"   ✓ DEZELFDE grenzen toegepast op test ({len(X_test_clean)} samples)")
    print("   Rationale: Extreme waarden kunnen tracking fouten zijn. Winsorization behoudt data maar beperkt invloed.")

# Strategie 2: Heart Rate - capping naar plausibele range (40 - 120 bpm)
if 'Heart Rate' in X_train_clean.columns:
    print("\n2. HEART RATE - Capping naar plausibele range (40 - 120 bpm)")
    print("-" * 40)
    
    # Medische plausibiliteitsgrenzen (domeinkennis)
    hr_lower, hr_upper = 40, 120
    
    # Toon originele ranges
    original_hr_min_train = X_train_clean['Heart Rate'].min()
    original_hr_max_train = X_train_clean['Heart Rate'].max()
    original_hr_min_test = X_test_clean['Heart Rate'].min()
    original_hr_max_test = X_test_clean['Heart Rate'].max()
    
    print(f"   TRAIN originele hartslag: {original_hr_min_train:.0f} - {original_hr_max_train:.0f} bpm")
    print(f"   TEST originele hartslag: {original_hr_min_test:.0f} - {original_hr_max_test:.0f} bpm")
    print(f"   Capping grenzen (medisch plausibel): {hr_lower} - {hr_upper} bpm")
    
    # Pas capping toe op TRAIN
    X_train_clean['Heart Rate'] = X_train_clean['Heart Rate'].clip(lower=hr_lower, upper=hr_upper)
    
    # Pas DEZELFDE grenzen toe op TEST
    X_test_clean['Heart Rate'] = X_test_clean['Heart Rate'].clip(lower=hr_lower, upper=hr_upper)
    
    print(f"   ✓ Capping toegepast op train ({len(X_train_clean)} samples)")
    print(f"   ✓ DEZELFDE grenzen toegepast op test ({len(X_test_clean)} samples)")
    print("   Rationale: Clipping behoudt observaties maar vermindert invloed van onrealistische uitschieters.")

# Strategie 3: Plausibiliteitscontroles (geen verwijdering, enkel rapportage)
print("\n3. ANDERE VARIABELEN - Plausibiliteitscheck (geen verwijdering)")
print("-" * 40)

# Sleep Duration (we behouden, want korte/lange slaap kan klinisch relevant zijn)
if 'Sleep Duration' in X_train_clean.columns:
    invalid_sleep_train = (X_train_clean['Sleep Duration'] < 0) | (X_train_clean['Sleep Duration'] > 24)
    invalid_sleep_test = (X_test_clean['Sleep Duration'] < 0) | (X_test_clean['Sleep Duration'] > 24)
    
    if invalid_sleep_train.sum() > 0 or invalid_sleep_test.sum() > 0:
        print(f"   ⚠ Sleep Duration: {invalid_sleep_train.sum()} train + {invalid_sleep_test.sum()} test waarden buiten 0-24 uur")
    else:
        print("   ✓ Sleep Duration: alle waarden binnen 0-24 uur (train + test)")

# BMI (optioneel)
if 'BMI' in X_train_clean.columns:
    invalid_bmi_train = (X_train_clean['BMI'] < 10) | (X_train_clean['BMI'] > 70)
    invalid_bmi_test = (X_test_clean['BMI'] < 10) | (X_test_clean['BMI'] > 70)
    
    if invalid_bmi_train.sum() > 0 or invalid_bmi_test.sum() > 0:
        print(f"   ⚠ BMI: {invalid_bmi_train.sum()} train + {invalid_bmi_test.sum()} test waarden buiten 10-70")
    else:
        print("   ✓ BMI waarden binnen plausibel bereik (train + test)")

# Age
if 'Age' in X_train_clean.columns:
    invalid_age_train = (X_train_clean['Age'] < 18) | (X_train_clean['Age'] > 100)
    invalid_age_test = (X_test_clean['Age'] < 18) | (X_test_clean['Age'] > 100)
    
    if invalid_age_train.sum() > 0 or invalid_age_test.sum() > 0:
        print(f"   ⚠ Age: {invalid_age_train.sum()} train + {invalid_age_test.sum()} test waarden buiten 18-100 jaar")
    else:
        print("   ✓ Age waarden binnen plausibel bereik (train + test)")

# ============================================================================
# STAP 8C: RESULTAAT VAN OUTLIER BEHANDELING
# ============================================================================
print("\n" + "-"*80)
print("RESULTAAT VAN OUTLIER BEHANDELING (CAPPING / WINSORIZATION)")
print("-"*80)

# We hebben geen rijen verwijderd; we hebben waarden gecapped/winsorized
rows_removed = 0
pct_removed = 0.0

print(f"\nTRAINING SET:")
print(f"  • Originele grootte: {X_train.shape[0]} rijen × {X_train.shape[1]} kolommen")
print(f"  • Na behandeling: {X_train_clean.shape[0]} rijen × {X_train_clean.shape[1]} kolommen")
print(f"  • Verwijderd: {rows_removed} rijen ({pct_removed:.2f}%)")

print(f"\nTEST SET:")
print(f"  • Originele grootte: {X_test.shape[0]} rijen × {X_test.shape[1]} kolommen")
print(f"  • Na behandeling: {X_test_clean.shape[0]} rijen × {X_test_clean.shape[1]} kolommen")
print(f"  • Verwijderd: {rows_removed} rijen ({pct_removed:.2f}%)")

print("\n✓ Geen rijen verwijderd - we hebben capping toegepast op extreme waarden")
print("✓ Outlier grenzen bepaald op TRAIN en toegepast op BEIDE sets (geen data leakage)")
print("✓ Extreme waarden beperkt via winsorization/clipping")

# Update de variabelen voor gebruik in volgende stappen
X_train = X_train_clean
X_test = X_test_clean

print("\n X_train en X_test zijn nu updated met outlier behandeling")
print("   Deze worden gebruikt in volgende stappen (encoding, scaling)")

In [ ]:
# ============================================================================
# STAP 9: FEATURE ENCODING - CATEGORISCHE VARIABELEN TRANSFORMEREN
# ============================================================================
print("\n" + "="*80)
print("STAP 9: FEATURE ENCODING - CATEGORISCHE VARIABELEN")
print("="*80)
print("""
WAAROM ENCODING NODIG IS:
=========================
Machine learning modellen werken met numerieke waarden. Categorische variabelen
zoals 'Gender' (Male/Female) of 'BMI Category' (Normal/Overweight/Obese) moeten
worden omgezet naar een numerieke representatie.

ENCODING METHODEN:
==================
1. LABEL ENCODING:
   • Ordinal categorieën: 'Low' → 0, 'Medium' → 1, 'High' → 2
   • Nadeel: impliceert orde/rangorde (2 > 1 > 0)
   • Gebruik alleen voor ordinale variabelen of target variabele

2. ONE-HOT ENCODING:
   • Nominale categorieën: 'Male' → [1,0], 'Female' → [0,1]
   • Voordeel: geen kunstmatige ordinale relatie
   • Nadeel: verhoogt dimensionaliteit (curse of dimensionality bij veel categorieën)
   • Drop_first=True om multicollineariteit te voorkomen (dummy variable trap)

3. NUMERIEKE SPLITS:
   • Blood Pressure: "120/80" → Systolic=120, Diastolic=80
   • Voorkomt explosie van dummy variabelen
   • Behoudt numerieke relatie

ONZE AANPAK MET TRAIN-TEST SPLIT:
==================================
CRITICAL: Data Leakage Preventie
---------------------------------
- One-hot encoding: fit op TRAIN, transform op TEST
- Label encoding (target): fit op TRAIN, transform op TEST
- Encoder leert categorieën alleen van train data

Waarom?
- Test set kan nieuwe categorieën hebben die train niet heeft gezien
- Encoder moet consistent zijn tussen train en test
- Voorkomt dat test informatie lekt naar train
""")

# ============================================================================
# STAP 9A: BLOOD PRESSURE SPLITS IN SYSTOLIC EN DIASTOLIC
# ============================================================================
print("\n" + "-"*80)
print("BLOOD PRESSURE TRANSFORMATIE")
print("-"*80)
print("""
PROBLEEM: Blood Pressure is categorisch met ~100+ unieke waarden
-----------------------------------------------------------------
Elke waarde zoals "120/80", "130/85", etc. zou een aparte dummy kolom krijgen.
Dit resulteert in 100+ features → curse of dimensionality!

OPLOSSING: Split in 2 numerieke features
-----------------------------------------
"120/80" → Systolic = 120, Diastolic = 80

Voordelen:
✓ Reduceert 100+ dummies naar 2 numerieke features
✓ Behoudt medische betekenis (systolische/diastolische druk)
✓ Modellen kunnen numerieke relaties leren
""")

def split_blood_pressure(df):
    """Split Blood Pressure kolom in Systolic en Diastolic"""
    df = df.copy()
    
    # Split "120/80" in twee delen
    bp_split = df['Blood Pressure'].str.split('/', expand=True)
    
    # Converteer naar integers
    df['Systolic'] = bp_split[0].astype(int)
    df['Diastolic'] = bp_split[1].astype(int)
    
    # Verwijder originele Blood Pressure kolom
    df = df.drop('Blood Pressure', axis=1)
    
    return df

print(f"\nVOOR transformatie:")
print(f"  • X_train shape: {X_train.shape}")
print(f"  • X_test shape: {X_test.shape}")
print(f"  • Blood Pressure unieke waarden in train: {X_train['Blood Pressure'].nunique()}")
print(f"  • Blood Pressure unieke waarden in test: {X_test['Blood Pressure'].nunique()}")

print("\nVoorbeeld Blood Pressure waarden:")
print(X_train['Blood Pressure'].head(10).tolist())

# Voer transformatie uit
X_train = split_blood_pressure(X_train)
X_test = split_blood_pressure(X_test)

print(f"\nNA transformatie:")
print(f"  • X_train shape: {X_train.shape}")
print(f"  • X_test shape: {X_test.shape}")

print("\nNieuwe kolommen toegevoegd:")
print(f"  • Systolic - Range train: [{X_train['Systolic'].min()}, {X_train['Systolic'].max()}]")
print(f"  • Systolic - Range test: [{X_test['Systolic'].min()}, {X_test['Systolic'].max()}]")
print(f"  • Diastolic - Range train: [{X_train['Diastolic'].min()}, {X_train['Diastolic'].max()}]")
print(f"  • Diastolic - Range test: [{X_test['Diastolic'].min()}, {X_test['Diastolic'].max()}]")

print("\nVoorbeeld eerste 5 rijen:")
print(X_train[['Systolic', 'Diastolic']].head())

print("\n✓ Blood Pressure succesvol gesplitst in 2 numerieke features")

# ============================================================================
# STAP 9B: IDENTIFICEER RESTERENDE CATEGORISCHE FEATURES
# ============================================================================
print("\n" + "-"*80)
print("IDENTIFICATIE VAN CATEGORISCHE FEATURES")
print("-"*80)

# Identificeer categorische kolommen in X_train
cat_cols_train = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"\nCategorische features in training set ({len(cat_cols_train)}):")
for col in cat_cols_train:
    unique_vals_train = X_train[col].nunique()
    unique_vals_test = X_test[col].nunique()
    print(f"  • {col}:")
    print(f"    - Train: {unique_vals_train} categorieën → {X_train[col].unique().tolist()}")
    print(f"    - Test: {unique_vals_test} categorieën → {X_test[col].unique().tolist()}")

# Check voor nieuwe categorieën in test set (kunnen problemen geven)
print("\n⚠️ CONTROLE: Nieuwe categorieën in test set?")
for col in cat_cols_train:
    train_cats = set(X_train[col].unique())
    test_cats = set(X_test[col].unique())
    new_cats = test_cats - train_cats
    
    if new_cats:
        print(f"  ⚠️ {col}: Test heeft nieuwe categorieën: {new_cats}")
        print(f"     → Deze worden behandeld als 'unknown' tijdens encoding")
    else:
        print(f"  ✓ {col}: Geen nieuwe categorieën in test")

# ============================================================================
# STAP 9C: ONE-HOT ENCODING VOOR FEATURES
# ============================================================================
print("\n" + "-"*80)
print("ONE-HOT ENCODING VOOR FEATURES")
print("-"*80)
print("""
METHODE: pd.get_dummies() met align
====================================
We gebruiken pandas get_dummies() omdat het eenvoudig is, maar we moeten
zorgen dat train en test dezelfde kolommen hebben na encoding.

Proces:
1. Encode train set → krijg dummy kolommen
2. Encode test set → krijg dummy kolommen
3. Align beide sets zodat ze identieke kolommen hebben
4. Missende kolommen in test worden gevuld met 0

Alternatief: sklearn OneHotEncoder met handle_unknown='ignore'
""")

print(f"\nVOOR encoding:")
print(f"  • X_train: {X_train.shape[0]} rijen × {X_train.shape[1]} kolommen")
print(f"  • X_test: {X_test.shape[0]} rijen × {X_test.shape[1]} kolommen")

# One-hot encoding op TRAIN
X_train_encoded = pd.get_dummies(X_train, columns=cat_cols_train, drop_first=True)

# One-hot encoding op TEST (met dezelfde kolommen)
X_test_encoded = pd.get_dummies(X_test, columns=cat_cols_train, drop_first=True)

print(f"\nNA encoding (voor align):")
print(f"  • X_train_encoded: {X_train_encoded.shape[0]} rijen × {X_train_encoded.shape[1]} kolommen")
print(f"  • X_test_encoded: {X_test_encoded.shape[0]} rijen × {X_test_encoded.shape[1]} kolommen")

# BELANGRIJK: Align zodat beide sets dezelfde kolommen hebben
# Kolommen die in train maar niet in test zitten → voeg toe aan test met 0
# Kolommen die in test maar niet in train zitten → verwijder uit test
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

print(f"\nNA align (train en test hebben nu IDENTIEKE kolommen):")
print(f"  • X_train_encoded: {X_train_encoded.shape[0]} rijen × {X_train_encoded.shape[1]} kolommen")
print(f"  • X_test_encoded: {X_test_encoded.shape[0]} rijen × {X_test_encoded.shape[1]} kolommen")

# Toon nieuwe kolommen
new_cols = [col for col in X_train_encoded.columns if col not in X_train.columns]
print(f"\nNieuwe dummy kolommen gecreëerd ({len(new_cols)}):")
for col in new_cols[:15]:  # Toon eerste 15
    print(f"  • {col}")
if len(new_cols) > 15:
    print(f"  ... en {len(new_cols) - 15} meer")

print("\n✓ One-hot encoding voltooid")
print("✓ Train en test hebben identieke kolommen")

# ============================================================================
# STAP 9D: LABEL ENCODING VOOR TARGET VARIABELE
# ============================================================================
print("\n" + "-"*80)
print("LABEL ENCODING VOOR TARGET VARIABELE")
print("-"*80)
print("""
Target encoding met sklearn LabelEncoder
=========================================
Process:
1. Fit encoder op y_train (leert de klassen)
2. Transform y_train → numerieke labels
3. Transform y_test met DEZELFDE encoder → numerieke labels

Dit garandeert consistente encoding tussen train en test.
""")

print(f"\nVOOR encoding:")
print(f"  • y_train: {y_train.shape[0]} samples, type: {y_train.dtype}")
print(f"  • y_test: {y_test.shape[0]} samples, type: {y_test.dtype}")
print(f"\nUnieke klassen in y_train: {sorted(y_train.unique())}")
print(f"Unieke klassen in y_test: {sorted(y_test.unique())}")

# Label encoding voor target
le = LabelEncoder()

# FIT op y_train (encoder leert de klassen van train data)
le.fit(y_train)

# TRANSFORM beide sets met gefitte encoder
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

# Converteer naar pandas Series voor consistentie
y_train_encoded = pd.Series(y_train_encoded, index=y_train.index, name='Sleep Disorder')
y_test_encoded = pd.Series(y_test_encoded, index=y_test.index, name='Sleep Disorder')

print(f"\nNA encoding:")
print(f"  • y_train_encoded: {y_train_encoded.shape[0]} samples, type: {y_train_encoded.dtype}")
print(f"  • y_test_encoded: {y_test_encoded.shape[0]} samples, type: {y_test_encoded.dtype}")

# Toon mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("\nLabel Encoding Mapping:")
for original, encoded in sorted(label_mapping.items(), key=lambda x: x[1]):
    count_train = (y_train_encoded == encoded).sum()
    count_test = (y_test_encoded == encoded).sum()
    print(f"  '{original}' → {encoded}")
    print(f"    - Train: {count_train} samples ({100*count_train/len(y_train_encoded):.1f}%)")
    print(f"    - Test: {count_test} samples ({100*count_test/len(y_test_encoded):.1f}%)")

print("\nRationale: Label encoding voor target is noodzakelijk voor sklearn classificatie.")
print("De numerieke waarden hebben geen ordinale betekenis in multiclass setting.")

# ============================================================================
# STAP 9E: VERIFICATIE EN SAMENVATTING
# ============================================================================
print("\n" + "-"*80)
print("VERIFICATIE EN SAMENVATTING")
print("-"*80)

print("\nEerste 5 rijen TRAINING set (features + target):")
display(pd.concat([X_train_encoded.head(), y_train_encoded.head()], axis=1))

print("\nEerste 5 rijen TEST set (features + target):")
display(pd.concat([X_test_encoded.head(), y_test_encoded.head()], axis=1))

print("\nKolomtypes in X_train_encoded:")
print(X_train_encoded.dtypes.value_counts())

print("\n" + "="*80)
print("✓ FEATURE ENCODING VOLTOOID")
print("="*80)
print("\nSamenvatting:")
print(f"  • Blood Pressure: Gesplitst in Systolic + Diastolic (2 numerieke features)")
print(f"  • Categorische features: {len(cat_cols_train)} → One-hot encoded")
print(f"  • Train features: {X_train.shape[1]} → {X_train_encoded.shape[1]} kolommen")
print(f"  • Test features: {X_test.shape[1]} → {X_test_encoded.shape[1]} kolommen")
print(f"  • Target variabele: Label encoded (3 klassen)")
print(f"  • Train samples: {X_train_encoded.shape[0]}")
print(f"  • Test samples: {X_test_encoded.shape[0]}")

print("\n BELANGRIJK: Encoder fit op TRAIN, transform op TEST")
print("   Dit voorkomt data leakage!")

print("\nReductie in features:")
print(f"  • Blood Pressure zou ~{X_train['Blood Pressure'].nunique() if 'Blood Pressure' in X_train.columns else 100} dummies geven")
print(f"  • Door splitsing: slechts 2 numerieke features (Systolic, Diastolic)")
print(f"  • Besparing: ~{100-2} features!")

# Update variabelen voor volgende stappen
X_train = X_train_encoded
X_test = X_test_encoded
y_train = y_train_encoded
y_test = y_test_encoded

print("\n✓ X_train, X_test, y_train, y_test zijn nu ge-encoded")
print("   Deze worden gebruikt in volgende stappen (scaling, modeling)")

# Check correlatie tussen Systolic en Diastolic
if 'Systolic' in X_train.columns and 'Diastolic' in X_train.columns:
    corr = X_train[['Systolic', 'Diastolic']].corr().iloc[0, 1]
    print(f"\n📊 CORRELATIE CHECK:")
    print(f"   Systolic ↔ Diastolic: r = {corr:.3f}")
    if abs(corr) > 0.9:
        print(f"   ⚠️ Hoge correlatie gedetecteerd (|r| > 0.9)")
        print(f"   → Overweeg één van beide te verwijderen om multicollineariteit te reduceren")
    else:
        print(f"   ✓ Correlatie acceptabel")

print("\n" + "="*80)
print("READY VOOR STAP 10: FEATURE SCALING")
print("="*80)

In [ ]:
# ============================================================================
# STAP 10: FEATURE SCALING - NORMALISATIE EN STANDAARDISATIE
# ============================================================================
print("\n" + "="*80)
print("STAP 10: FEATURE SCALING (STANDARDISATIE)")
print("="*80)

print("""
WAAROM FEATURE SCALING NODIG IS:
=================================

Variabelen hebben verschillende schalen:
- Daily Steps: 0 - 10,000+ 
- Quality of Sleep: 1 - 10
- Heart Rate: 50 - 100 bpm

PROBLEEM ZONDER SCALING:
========================
- Distance-based modellen (KNN) worden gedomineerd door features met grote schaal
- Gradient descent convergeert langzamer
- Regularisatie (L1/L2) werkt niet eerlijk tussen features

METHODEN:
=========

1. MIN-MAX NORMALISATIE (0-1 range):
   x_scaled = (x - x_min) / (x_max - x_min)
   • Voordeel: Behoudt distributie shape
   • Nadeel: Gevoelig voor outliers

2. STANDARDISATIE (Z-SCORE):
   x_scaled = (x - μ) / σ
   • Voordeel: Robuust tegen outliers, mean=0 en std=1
   • Nadeel: Geen vaste min/max
   • BEST PRACTICE voor de meeste ML algoritmen

ONZE KEUZE: StandardScaler (Z-score normalisatie)
===================================================
- Geschikt voor variabelen met outliers (hebben we behandeld in stap 8)
- Werkt goed met tree-based modellen én lineaire modellen
- Industrie standaard in sklearn pipelines

CRITICAL: DATA LEAKAGE PREVENTIE
=================================
- Scaler fit op TRAIN data (leert mean en std van train)
- Scaler transform op TRAIN data (past train statistics toe)
- Scaler transform op TEST data (past TRAIN statistics toe op test!)
- Test set blijft "ongezien" - we leren NIETS van test data

Waarom?
- Als we mean/std berekenen op hele dataset, dan lekt test info naar train
- Model performance zou te optimistisch zijn
- In productie hebben we ook geen toegang tot test statistics
""")

print("\n" + "-"*80)
print("HUIDIGE DATA STATUS")
print("-"*80)

print(f"\nTRAINING SET:")
print(f"  • X_train shape: {X_train.shape[0]} samples × {X_train.shape[1]} features")
print(f"  • y_train shape: {y_train.shape[0]} samples")

print(f"\nTEST SET:")
print(f"  • X_test shape: {X_test.shape[0]} samples × {X_test.shape[1]} features")
print(f"  • y_test shape: {y_test.shape[0]} samples")

print(f"\nFeature kolommen ({len(X_train.columns)}):")
# Toon eerste 15 kolommen
cols_to_show = X_train.columns.tolist()[:15]
for col in cols_to_show:
    print(f"  • {col}")
if len(X_train.columns) > 15:
    print(f"  ... en {len(X_train.columns) - 15} meer kolommen")

# ============================================================================
# STAP 10A: SCALING OP TRAINING SET
# ============================================================================
print("\n" + "-"*80)
print("STAP 10A: STANDARDISATIE OP TRAINING SET")
print("-"*80)

# Toon voorbeeld van niet-geschaalde data (TRAIN)
print("\nVoorbeeld TRAIN features VOOR scaling:")
display(X_train.head())

print("\nDescriptive statistics TRAIN VOOR scaling:")
display(X_train.describe())

# Initialiseer scaler
scaler = StandardScaler()

# FIT scaler op TRAIN data (leert mean en std van train)
scaler.fit(X_train)

print("\n Scaler Statistics (geleerd van TRAIN data):")
print(f"  • Mean per feature (eerste 5): {scaler.mean_[:5]}")
print(f"  • Std per feature (eerste 5): {scaler.scale_[:5]}")

# TRANSFORM train data met gefitte scaler
X_train_scaled = scaler.transform(X_train)

# Converteer terug naar DataFrame voor visualisatie
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

print("\nVoorbeeld TRAIN features NA scaling:")
display(X_train_scaled.head())

print("\nDescriptive statistics TRAIN NA scaling:")
display(X_train_scaled.describe())

print("""
INTERPRETATIE TRAIN:
====================
- Mean ≈ 0 (kleine floating point errors acceptabel)
- Std ≈ 1 voor alle features
- Dit is verwacht omdat we scaler gefitted hebben op train data
""")

# ============================================================================
# STAP 10B: SCALING OP TEST SET (MET TRAIN STATISTICS)
# ============================================================================
print("\n" + "-"*80)
print("STAP 10B: STANDARDISATIE OP TEST SET (MET TRAIN STATISTICS)")
print("-"*80)

print("""
BELANGRIJK: We gebruiken GEEN fit() op test data!
==================================================

We gebruiken de TRAIN mean en std om test data te schalen.
Dit betekent dat test data NIET per se mean=0 en std=1 zal hebben.

Dit is CORRECT gedrag:
- In productie hebben we ook nieuwe data die we schalen met train statistics
- Test set moet "ongezien" blijven
- Kleine afwijkingen van mean=0/std=1 in test zijn normaal en verwacht
""")

# Toon voorbeeld van niet-geschaalde data (TEST)
print("\nVoorbeeld TEST features VOOR scaling:")
display(X_test.head())

print("\nDescriptive statistics TEST VOOR scaling:")
display(X_test.describe())

# TRANSFORM test data met TRAIN scaler (GEEN fit!)
X_test_scaled = scaler.transform(X_test)

# Converteer terug naar DataFrame voor visualisatie
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("\nVoorbeeld TEST features NA scaling:")
display(X_test_scaled.head())

print("\nDescriptive statistics TEST NA scaling:")
display(X_test_scaled.describe())

print("""
INTERPRETATIE TEST:
===================
- Mean kan AFWIJKEN van 0 (bijvoorbeeld -0.15 of +0.20)
- Std kan AFWIJKEN van 1 (bijvoorbeeld 0.95 of 1.08)
- Dit is NORMAAL en CORRECT - test data is geschaald met TRAIN statistics
- Kleine afwijkingen zijn verwacht als train en test distributie iets verschillen
""")

# ============================================================================
# STAP 10C: VERIFICATIE EN SAMENVATTING
# ============================================================================
print("\n" + "-"*80)
print("VERIFICATIE VAN SCALING")
print("-"*80)

# Controleer of alle features geschaald zijn
print("\nControle: Zijn alle features geschaald?")
print("\nTRAIN set - Mean en Std per feature (eerste 10):")
train_stats = pd.DataFrame({
    'Mean': X_train_scaled.mean(),
    'Std': X_train_scaled.std()
}).head(10)
display(train_stats)

print("\nTEST set - Mean en Std per feature (eerste 10):")
test_stats = pd.DataFrame({
    'Mean': X_test_scaled.mean(),
    'Std': X_test_scaled.std()
}).head(10)
display(test_stats)

print("\n✓ TRAIN: Mean ≈ 0, Std ≈ 1 (exact, want scaler gefitted op train)")
print("⚠️ TEST: Mean en Std kunnen afwijken (normaal, want geschaald met train statistics)")

# ============================================================================
# STAP 10D: VISUALISATIE VAN SCALING EFFECT
# ============================================================================
print("\n" + "-"*80)
print("VISUALISATIE: VOOR vs NA SCALING")
print("-"*80)

# Selecteer een paar features om te visualiseren
features_to_plot = [col for col in ['Sleep Duration', 'Heart Rate', 'Daily Steps', 'BMI', 'Age'] 
                    if col in X_train.columns][:4]

if len(features_to_plot) > 0:
    fig, axes = plt.subplots(2, len(features_to_plot), figsize=(16, 8))
    
    for i, feature in enumerate(features_to_plot):
        # Voor scaling (train)
        axes[0, i].hist(X_train[feature], bins=20, edgecolor='black', alpha=0.7)
        axes[0, i].set_title(f'{feature}\nVOOR scaling (Train)')
        axes[0, i].set_xlabel('Waarde')
        axes[0, i].set_ylabel('Frequentie')
        
        # Na scaling (train)
        axes[1, i].hist(X_train_scaled[feature], bins=20, edgecolor='black', alpha=0.7, color='green')
        axes[1, i].set_title(f'{feature}\nNA scaling (Train)')
        axes[1, i].set_xlabel('Z-score')
        axes[1, i].set_ylabel('Frequentie')
        axes[1, i].axvline(0, color='red', linestyle='--', label='Mean=0')
    
    plt.suptitle('Effect van Standardisatie op Features (Training Set)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Geen numerieke features gevonden om te plotten")

print("\n" + "="*80)
print("✓ FEATURE SCALING VOLTOOID")
print("="*80)

print("\nSamenvatting:")
print(f"  • Scaling methode: StandardScaler (Z-score normalisatie)")
print(f"  • Scaler fitted op: TRAIN data ({X_train.shape[0]} samples)")
print(f"  • Train features geschaald: {X_train_scaled.shape[1]} features")
print(f"  • Test features geschaald: {X_test_scaled.shape[1]} features")
print(f"  • Train mean ≈ 0, std ≈ 1: ✓")
print(f"  • Test geschaald met TRAIN statistics: ✓")
print(f"  • Data leakage voorkomen: ✓")



# Update variabelen voor volgende stappen
X_train = X_train_scaled
X_test = X_test_scaled

print("\n✓ X_train en X_test zijn nu geschaald en klaar voor modeling")

In [ ]:
# ============================================================================
# STAP 11: FEATURE SELECTION & ENGINEERING - OVERWEGINGEN
# ============================================================================
print("\n" + "="*80)
print("STAP 11: FEATURE SELECTION & ENGINEERING - OVERWEGINGEN")
print("="*80)

print("""
HUIDIGE FEATURES:
=================

We hebben alle beschikbare features behouden na encoding:
- Numerieke features: Sleep Duration, Quality of Sleep, Stress Level, etc.
- One-hot encoded categorische features: Gender, BMI Category, Occupation


WAAROM BEHOUDEN WE ALLE FEATURES?
==================================

Voor de BASELINE modellen gebruiken we bewust ALLE features, ook al hebben we
in stap 7 (EDA) multicollineariteit gedetecteerd:

- Systolic ↔ Diastolic: r = 0.979 (extreem hoog)
- Physical Activity ↔ Daily Steps: r = 0.761 (hoog)
- Sleep Duration ↔ Quality of Sleep: r = 0.847 (hoog)

Rationale voor behouden:
1. Data-driven benadering: Laat MODEL bepalen welke features belangrijk zijn
2. Wetenschappelijke methode: Train eerst, analyseer dan
3. Vergelijkingsbasis: Baseline (alle features) vs Optimized (geselecteerde features)
4. Objectiviteit: Geen vooringenomen keuzes zonder bewijs

FEATURE SELECTION STRATEGIE - IN ML FASE:
==========================================

In de "Uitwerking ML Vraagstuk" fase zullen we:

STAP 1: BASELINE MODELS
  → Train Random Forest, XGBoost, Logistic Regression met ALLE 23 features
  → Evalueer performance en feature importance

STAP 2: FEATURE IMPORTANCE ANALYSE
  → Analyseer feature importance uit tree-based modellen
  → Identificeer features met <1% importance
  → Analyseer correlaties in context van model performance
  
STAP 3: MULTICOLLINEARITEIT BEHANDELING
  ⚠️ DIASTOLIC VERWIJDERING:
     • Systolic vs Diastolic (r=0.979)
     • Keuze: Behoud Systolic (klinisch belangrijker voor cardiovasculaire risico)
     • Rationale: 95.8% overlap in informatie, geen toegevoegde voorspellende waarde
  
  💡 OVERIGE CORRELATIES:
     • Physical Activity vs Daily Steps (r=0.761)
       → Beide behouden in baseline, evalueer feature importance
     • Sleep Duration vs Quality (r=0.847)
       → Conceptueel verschillend (objectief vs subjectief), beide behouden

STAP 4: OPTIMIZED MODELS
  → Retrain modellen met geselecteerde features
  → Vergelijk: Baseline (23 features) vs Optimized (≈20-22 features)
  → Evalueer: Accuracy, precision, recall, F1-score
  → Besluit: Behoud optimized model als performance gelijk of beter

STAP 5: FEATURE ENGINEERING (OPTIONEEL)
  Indien baseline performance onvoldoende:
  • Sleep Efficiency = (Sleep Duration / 8) × Quality of Sleep
  • Activity-Stress Balance = Physical Activity / (Stress Level + 1)
  • BMI-Age Interaction = BMI × (Age / 50)
  • High Risk Binary = (BMI > 30) & (Age > 50) & (Heart Rate > 80)

SAMPLES/FEATURES RATIO ANALYSE:
================================

Huidige ratio: 80,000 samples / 23 features = 3,478:1

✓ UITSTEKEND! Vuistregels:
  • Minimum ratio: 10:1 (wij hebben 3,478:1)
  • Ideaal voor complex models: >100:1 (wij hebben dit!)
  • Geen curse of dimensionality issues
  • Voldoende data voor stabiele feature importance estimates

ONZE AANPAK: "START SIMPEL, ITEREER OP BASIS VAN DATA"
=======================================================

1. Preprocessing: Behoud alle features (huidige fase)
2. Baseline: Train met alle features
3. Analyse: Feature importance + multicollineariteit impact
4. Optimize: Verwijder redundante/onbelangrijke features
5. Compare: Baseline vs Optimized performance
6. Decide: Kies beste model op basis van metrics

Dit is de INDUSTRIE STANDAARD aanpak en volgt wetenschappelijke methode.
""")

print("\n" + "-"*80)
print("FINALE FEATURE SET VOOR BASELINE MODELING")
print("-"*80)

print(f"\nTRAINING SET:")
print(f"  • Samples: {X_train.shape[0]:,}")
print(f"  • Features: {X_train.shape[1]}")
print(f"  • Samples/Features ratio: {X_train.shape[0] / X_train.shape[1]:,.1f}:1")

print(f"\nTEST SET:")
print(f"  • Samples: {X_test.shape[0]:,}")
print(f"  • Features: {X_test.shape[1]}")

print(f"\nFeature lijst ({X_train.shape[1]} features):")

# Groepeer features
original_numeric = [col for col in X_train.columns if not any(
    prefix in col for prefix in ['Gender_', 'Occupation_', 'BMI Category_']
)]
gender_features = [col for col in X_train.columns if col.startswith('Gender_')]
occupation_features = [col for col in X_train.columns if col.startswith('Occupation_')]
bmi_cat_features = [col for col in X_train.columns if col.startswith('BMI Category_')]

print("\n1. NUMERIEKE FEATURES:")
for i, feature in enumerate(original_numeric, 1):
    print(f"   {i:2d}. {feature}")

if gender_features:
    print(f"\n2. GENDER FEATURES ({len(gender_features)}):")
    for i, feature in enumerate(gender_features, 1):
        print(f"   {i:2d}. {feature}")

if occupation_features:
    print(f"\n3. OCCUPATION FEATURES ({len(occupation_features)}):")
    for i, feature in enumerate(occupation_features, 1):
        print(f"   {i:2d}. {feature}")

if bmi_cat_features:
    print(f"\n4. BMI CATEGORY FEATURES ({len(bmi_cat_features)}):")
    for i, feature in enumerate(bmi_cat_features, 1):
        print(f"   {i:2d}. {feature}")

print("\n" + "-"*80)
print("DATA QUALITY FINAL CHECK")
print("-"*80)

print(f"\n✓ Missing values: {X_train.isnull().sum().sum()} (train) + {X_test.isnull().sum().sum()} (test)")
print(f"✓ Infinite values: {np.isinf(X_train.values).sum()} (train) + {np.isinf(X_test.values).sum()} (test)")
print(f"✓ Data types: All numeric ({X_train.select_dtypes(include=[np.number]).shape[1]}/{X_train.shape[1]} features)")
print(f"✓ Scaling: Completed (StandardScaler, fit on train)")
print(f"✓ Encoding: Completed (One-hot + Label encoding)")
print(f"✓ Outliers: Treated (conservative capping)")
print(f"✓ Data leakage: Prevented (fit train → transform test)")

print("\n" + "="*80)
print("✓ DATA PREPROCESSING PIPELINE VOLTOOID")
print("="*80)

print("""
🚀 DATA IS KLAAR VOOR MODEL TRAINING!
""")
